# Lab2: Backpropagation handmade

In [94]:
# 1. Build the class for math expressions
# 2. Build some basic operations
# 3. Build computation graph
# 4. Build some activations
# 5. Build backpropagation

In [95]:
import numpy as np
from collections import defaultdict

import math
import matplotlib.pyplot as plt

In [96]:
def print_testpr(l):
    for i in l:
        print(i)

In [97]:
from graphlib import TopologicalSorter
class Parameter:
    def __init__(self, value: float, name: str) -> None:
        self._value = value
        self._name = name

        self._grad = 0.0
        self._backward = lambda: None
        
        self.parameters = dict()
        self.parameters[name] = self
        self._graph = defaultdict(set)
        self._graph[self._name] = set()

    def __repr__(self) -> str:
        return f"Parameter {self._name} = {self._value}; dL/d[{self._name}] = {self._grad}"

    def __mul__(self, other):
        result = Parameter(
            self._value * other._value,
            f'{self._name} * {other._name}'
        )
        
        result.parameters.update(self.parameters)
        result.parameters.update(other.parameters)
        result.parameters[result._name] = result
        
        result._graph.update(self._graph)
        result._graph.update(other._graph)
        result._graph[result._name].add(self._name)
        result._graph[result._name].add(other._name)

        def _backward():
            self._grad += other._value * result._grad #dL / dself
            other._grad += self._value * result._grad # dL / dother

        result._backward = _backward

        return result

    def __add__(self, other):
        result = Parameter(
            self._value + other._value,
            f'[{self._name} + {other._name}]'
        )
        
        result.parameters.update(self.parameters)
        result.parameters.update(other.parameters)
        result.parameters[result._name] = result
        
        result._graph.update(self._graph)
        result._graph.update(other._graph)
        result._graph[result._name].add(self._name)
        result._graph[result._name].add(other._name)

        def _backward():
            self._grad += 1.0 * result._grad  #dL / dself
            other._grad += 1.0 * result._grad # dL / dother

        result._backward = _backward

        return result

    def sigmoid(self):
        # f(x) = 1 / (1 + exp(self._value))
        # f'(x) = f(x) * (1 - f(x))

        val = 1.0 / (1.0 + math.exp(-self._value))

        result = Parameter(
            val,
            f"σ({self._name})"
        )
        
        result.parameters.update(self.parameters)
        result.parameters[result._name] = result
        
        result._graph.update(self._graph)
        result._graph[result._name].add(self._name)
        
        def _backward():
            self._grad = result._grad * val * (1 - val)

        result._backward = _backward

        return result

    def _topological_sort(self):
        in_degree = {node: 0 for node in self._graph}
        for node in self._graph:
            for neighbor in self._graph[node]:
                in_degree[neighbor] += 1

        queue = [node for node in self._graph if in_degree[node] == 0]
        result = []

        while queue:
            current_node = queue.pop(0)
            result.append(current_node)

            for neighbor in self._graph[current_node]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    queue.append(neighbor)

        return result

    def backward(self):
        queue = self._topological_sort()
        print(queue)
        self._grad = 1
        for i in queue:
            print(i)
            self.parameters[i]._backward()
            
            


    def SiLU(self):
        # LAB 2 TASK 2
        # f(x) = x*sigma(x)
        # f'(x) = sigma(x) + x*sigma(x)*(1-sigma(x))
        val = self._value * self.sigmoid()._value

        result = Parameter(
            val,
            f"{self._name}*σ({self._name})"
        )
        
        result.parameters.update(self.parameters)
        result.parameters[result._name] = result
        result._graph.update(self._graph)
        result._graph[result._name].add(self._name)
        
        def _backward():
            self._grad = result._grad * ( self.sigmoid()._value + self._value*self.sigmoid()._value*(1-self.sigmoid()._value) )

        result._backward = _backward

        return result
        

    def ReLU(self):
        # LAB 2 TASK 2
        # f(x) = max(0, x)
        # f'(x) = z>0
        val = max(0, self._value)

        result = Parameter(
            val,
            f"max(0, {self._name})"
        )
        
        result.parameters.update(self.parameters)
        result.parameters[result._name] = result
        result._graph.update(self._graph)
        result._graph[result._name].add(self._name)
        
        def _backward():
            self._grad = result._grad * ( self._value > 0 )

        result._backward = _backward

        return result

In [98]:
def sgd(parameters: list, learning_rate = 0.3): 
    for j in parameters:
        j._value -= learning_rate * j._grad        
    return parameters

## Task 1

### Test 1

In [99]:
a = Parameter(3.0, 'a')
b = Parameter(2.0, 'b')
c = Parameter(5.0, 'c')
d = Parameter(5.0, 'd')

In [100]:
print(a)
print(b)
print(c)
print(d)

Parameter a = 3.0; dL/d[a] = 0.0
Parameter b = 2.0; dL/d[b] = 0.0
Parameter c = 5.0; dL/d[c] = 0.0
Parameter d = 5.0; dL/d[d] = 0.0


In [101]:
u = a * b
v = u + c
L = v * d

In [102]:
L.backward()

['[a * b + c] * d', 'd', '[a * b + c]', 'a * b', 'c', 'b', 'a']
[a * b + c] * d
d
[a * b + c]
a * b
c
b
a


In [103]:
l1 = [ L, v, d, u, c, a, b]
print_testpr(l1)

Parameter [a * b + c] * d = 55.0; dL/d[[a * b + c] * d] = 1
Parameter [a * b + c] = 11.0; dL/d[[a * b + c]] = 5.0
Parameter d = 5.0; dL/d[d] = 11.0
Parameter a * b = 6.0; dL/d[a * b] = 5.0
Parameter c = 5.0; dL/d[c] = 5.0
Parameter a = 3.0; dL/d[a] = 10.0
Parameter b = 2.0; dL/d[b] = 15.0


### Test 2

In [104]:
y1 = Parameter(3.0, 'y1')
y2 = Parameter(2.0, 'y2')
c1 = Parameter(7.0, 'c1')
c2 = Parameter(5.0, 'c2')

In [105]:
y1c1 = y1*c1
y2c2y1 = y2*c2*y1
our = y1c1 + y2c2y1

In [106]:
our.backward()

['[y1 * c1 + y2 * c2 * y1]', 'y1 * c1', 'y2 * c2 * y1', 'c1', 'y2 * c2', 'y1', 'y2', 'c2']
[y1 * c1 + y2 * c2 * y1]
y1 * c1
y2 * c2 * y1
c1
y2 * c2
y1
y2
c2


In [107]:
test = [our, y2c2y1, y1c1, y1, y2, c1, c2]
print_testpr(test)

Parameter [y1 * c1 + y2 * c2 * y1] = 51.0; dL/d[[y1 * c1 + y2 * c2 * y1]] = 1
Parameter y2 * c2 * y1 = 30.0; dL/d[y2 * c2 * y1] = 1.0
Parameter y1 * c1 = 21.0; dL/d[y1 * c1] = 1.0
Parameter y1 = 3.0; dL/d[y1] = 17.0
Parameter y2 = 2.0; dL/d[y2] = 15.0
Parameter c1 = 7.0; dL/d[c1] = 3.0
Parameter c2 = 5.0; dL/d[c2] = 6.0


#### Test 3

In [108]:
x1 = Parameter(3.0, 'x1')
x2 = Parameter(4.0, 'x2')

w1 = Parameter(1.0, 'w1')
w2 = Parameter(2.0, 'w2')

x1w1 = x1 * w1
x2w2 = x2 * w2
xw = x1w1 + x2w2
out = xw.sigmoid()

In [109]:
out.backward()

['σ([x1 * w1 + x2 * w2])', '[x1 * w1 + x2 * w2]', 'x2 * w2', 'x1 * w1', 'x2', 'w2', 'x1', 'w1']
σ([x1 * w1 + x2 * w2])
[x1 * w1 + x2 * w2]
x2 * w2
x1 * w1
x2
w2
x1
w1


In [110]:
l2 = [out, xw, x2w2, x1w1, x1, x2, w1, w2]
print_testpr(l2)

Parameter σ([x1 * w1 + x2 * w2]) = 0.999983298578152; dL/d[σ([x1 * w1 + x2 * w2])] = 1
Parameter [x1 * w1 + x2 * w2] = 11.0; dL/d[[x1 * w1 + x2 * w2]] = 1.670114291046157e-05
Parameter x2 * w2 = 8.0; dL/d[x2 * w2] = 1.670114291046157e-05
Parameter x1 * w1 = 3.0; dL/d[x1 * w1] = 1.670114291046157e-05
Parameter x1 = 3.0; dL/d[x1] = 1.670114291046157e-05
Parameter x2 = 4.0; dL/d[x2] = 3.340228582092314e-05
Parameter w1 = 1.0; dL/d[w1] = 5.010342873138471e-05
Parameter w2 = 2.0; dL/d[w2] = 6.680457164184628e-05


#### Test 4

In [111]:
xx = x1*x1
xx.backward()

['x1 * x1', 'x1']
x1 * x1
x1


In [112]:
xx, x1

(Parameter x1 * x1 = 9.0; dL/d[x1 * x1] = 1,
 Parameter x1 = 3.0; dL/d[x1] = 6.00001670114291)

#### Test 5

In [113]:
x = Parameter(4, "x")
y = Parameter(5, "y")
Lout = x*(x*x+y)

In [120]:
Lout.backward()

['x * [x * x + y]', '[x * x + y]', 'x * x', 'y', 'x']
x * [x * x + y]
[x * x + y]
x * x
y
x
x


In [115]:
test2 = [Lout, x, y]
print_testpr(test2)

Parameter x * [x * x + y] = 84; dL/d[x * [x * x + y]] = 1
Parameter x = 4; dL/d[x] = 53.0
Parameter y = 5; dL/d[y] = 4.0


## Task 2-3

In [92]:
N = 20

In [93]:
x = Parameter(3.0, 'x')
b = Parameter(1.0, 'b')

w1 = Parameter(1.0, 'w1')
w2 = Parameter(2.0, 'w2')
parameters = [w1, w2, b]
L = (w2*(w1*x+b).ReLU()+b).ReLU()
print("Before:")
print(L)
print_testpr(parameters)
for i in range(N):
    L.backward()
    sgd(parameters)
    L = (w2*(w1*x+b).ReLU()+b).ReLU()
print("After:")
print(L)
print_testpr(parameters)

Before:
Parameter max(0, [w2 * max(0, [w1 * x + b]) + b]) = 9.0; dL/d[max(0, [w2 * max(0, [w1 * x + b]) + b])] = 0.0
Parameter w1 = 1.0; dL/d[w1] = 0.0
Parameter w2 = 2.0; dL/d[w2] = 0.0
Parameter b = 1.0; dL/d[b] = 0.0
['max(0, [w2 * max(0, [w1 * x + b]) + b])', '[w2 * max(0, [w1 * x + b]) + b]', 'w2 * max(0, [w1 * x + b])', 'max(0, [w1 * x + b])', 'w2', '[w1 * x + b]', 'b', 'w1 * x', 'x', 'w1']
max(0, [w2 * max(0, [w1 * x + b]) + b])
[w2 * max(0, [w1 * x + b]) + b]
w2 * max(0, [w1 * x + b])
max(0, [w1 * x + b])
w2
[w1 * x + b]
b
w1 * x
x
w1
['max(0, [w2 * max(0, [w1 * x + b]) + b])', '[w2 * max(0, [w1 * x + b]) + b]', 'w2 * max(0, [w1 * x + b])', 'max(0, [w1 * x + b])', 'w2', '[w1 * x + b]', 'b', 'w1 * x', 'x', 'w1']
max(0, [w2 * max(0, [w1 * x + b]) + b])
[w2 * max(0, [w1 * x + b]) + b]
w2 * max(0, [w1 * x + b])
max(0, [w1 * x + b])
w2
[w1 * x + b]
b
w1 * x
x
w1
['max(0, [w2 * max(0, [w1 * x + b]) + b])', '[w2 * max(0, [w1 * x + b]) + b]', 'w2 * max(0, [w1 * x + b])', 'max(0, [w1 * 